# Boosting 算法选股

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ_ML

d:\Anaconda\envs\py38\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# 策略

In [38]:
class ml(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.datas[0].datetime.date(0))
        if self.datas[0].datetime.date(0).month == 3:
            print(self.datas[0].ROE[0])
        else:
            print(self.datas[0].ROE[0] - self.datas[0].ROE[-1])


    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

In [5]:
class PandasData_ML(bt.feeds.PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = ('EPS','NetProfit','ROE', )

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', None),
        ('volume', None),
        ('openinterest', None),
        ('EPS', -1),
        ('NetProfit', -1),
        ('ROE', -1),
    )

In [3]:
s_date = datetime.datetime.now()- datetime.timedelta(days=385*2)
e_date = datetime.datetime.now()
stock_code = '002142.SZ'
utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',stock_code,
                        s_date, e_date)

,eps,netprofit_margin,roe
trade_date,,,
2020-09-30,1.93,37.5500,10.4552
2020-12-31,2.43,36.8174,13.7576
2021-03-31,0.79,35.8309,3.9179
2021-06-30,1.58,38.2070,7.6970
2021-09-30,2.38,36.8975,11.4043
2021-12-31,3.13,37.1566,14.5918
2022-03-31,0.87,37.4566,3.7539
2022-06-30,1.71,38.3483,7.2564


In [39]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(ml, printlog=True) 

    s_date = datetime.datetime.now()- datetime.timedelta(days=365*2)
    e_date = datetime.datetime.now()

    # Create stock Data Feed
    stock_index = '002142.SZ'
    df = utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',stock_code,
                        s_date, e_date)
    #df = pd.read_csv('C:\\Users\\a\\Desktop\\Backtrader\\Data\\2019\\' + stock_index + '.csv', 
    #                converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = PandasData_ML(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    #cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
2020-12-31
6.5012
2021-03-31
3.9179
2021-06-30
3.7791
2021-09-30
3.707299999999999
2021-12-31
3.1875
2022-03-31
3.7539
2022-06-30
3.5025000000000004
Final Portfolio Value: 20000.00
Net Profit: 0.00%


In [27]:
s_date = datetime.datetime.now()- datetime.timedelta(days=365*2)
e_date = datetime.datetime.now()

# Create stock Data Feed
stock_index = '002142.SZ'
utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',stock_code,s_date, e_date)

,eps,netprofit_margin,roe
trade_date,,,
2020-12-31,2.43,36.8174,13.7576
2021-03-31,0.79,35.8309,3.9179
2021-06-30,1.58,38.2070,7.6970
2021-09-30,2.38,36.8975,11.4043
2021-12-31,3.13,37.1566,14.5918
2022-03-31,0.87,37.4566,3.7539
2022-06-30,1.71,38.3483,7.2564


# 选股

In [13]:
s_date = datetime.datetime.now()- datetime.timedelta(days=365*2)
e_date = datetime.datetime.now()

pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
stock_list = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code').ts_code

roe_list = {}
for s in stock_list:
    df = utilsJ_ML.stock_finance('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',s,s_date, e_date)
    df.reset_index(inplace=True)
    try:
        df['roe_seasonal'] = df.roe_dt.diff()
        df.loc[[x.month == 3 for x in df.trade_date],'roe_seasonal'] = df[[x.month == 3 for x in df.trade_date]]['roe_dt']
        if df.roe_seasonal.iloc[-1] > df.roe_seasonal.iloc[-2] and df.roe_seasonal.iloc[-2] > df.roe_seasonal.iloc[-3] and (df.roe_seasonal.iloc[-1] + df.roe_seasonal.iloc[-2]) > (df.roe_seasonal.iloc[-3] + df.roe_seasonal.iloc[-4]):
            roe_list[s] = df.roe_dt.iloc[-1]
    except:
        print(s)
        print(df)
    time.sleep(0.8)
print(sorted(roe_list.items(), key=lambda x:x[1]))

000820.SZ
  trade_date     eps  netprofit_margin  roe_dt  roe_seasonal
0 2022-03-31 -0.0043          -50.9821 -3.3308       -3.3308
1 2022-06-30 -0.0049          -22.1503 -3.3218        0.0090
002175.SZ
  trade_date     eps  netprofit_margin  roe_dt  roe_seasonal
0 2022-03-31  0.0068           13.1068  2.4473        2.4473
1 2022-06-30  0.0216           19.7292  5.9807        3.5334
002766.SZ
  trade_date     eps  netprofit_margin  roe_dt  roe_seasonal
0 2022-03-31 -0.0046           -2.1016 -0.7380       -0.7380
1 2022-06-30  0.0020            0.4133  0.0675        0.8055
301097.SZ
Empty DataFrame
Columns: [trade_date, eps, netprofit_margin, roe_dt, roe_seasonal]
Index: []
600198.SH
  trade_date     eps  netprofit_margin   roe_dt  roe_seasonal
0 2022-03-31 -0.0330          -25.8083 -19.1450      -19.1450
1 2022-06-30 -0.0151            4.3390 -21.2946       -2.1496
600225.SH
  trade_date   eps  netprofit_margin  roe_dt  roe_seasonal
0 2022-03-31 -0.02         -251.8881 -4.4262       -4

In [15]:
result = sorted(roe_list.items(), key=lambda x:x[1])[-int(np.round(len(roe_list) * 0.2)):-1]

In [19]:
pd.DataFrame({'股票代码':[x[0] for x in result], '2022年二季度ROE': [x[1] for x in result]}).to_csv('全市场ROE报告.csv')